In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# --- 1. 데이터 불러오기 ---
# ❗️ 사용자 수정 필요: 실제 엑셀 파일 경로로 변경하세요.
excel_path = 'C:\workspace\키움뱅크\ML\dart_credit_risk_result.xlsx'
try:
    df = pd.read_excel(excel_path)
except FileNotFoundError:
    print(f"오류: '{excel_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    exit()

print("--- 데이터 일부 확인 ---")
print(df.head())
print("\n--- 데이터 정보 ---")
df.info()


# --- 2. 데이터 전처리 ---
# ❗️ 사용자 수정 필요: 실제 타겟 변수(예측하려는 값)의 열 이름으로 변경하세요.
target_column = 'Grade'

# 'corp_code'는 회사를 식별하는 고유 ID일 가능성이 높으므로, 모델 학습에서 제외합니다.
# 만약 중요한 feature라면 이 줄을 주석 처리하세요.
if 'corp_code' in df.columns:
    df = df.drop('corp_code', axis=1)

# X (독립 변수, Features)와 y (종속 변수, Target) 분리
X = df.drop(target_column, axis=1)
y = df[target_column]

# 범주형 데이터(문자열)를 숫자로 변환 (One-Hot Encoding)
X_encoded = pd.get_dummies(X, drop_first=True)

# 타겟 변수(신용등급)도 숫자(0, 1, 2...)로 변환 (Label Encoding)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 인코딩 후의 열 이름을 저장 (새로운 데이터 예측 시 사용)
encoded_columns = X_encoded.columns

print("\n--- 원-핫 인코딩 후 데이터 일부 확인 ---")
print(X_encoded.head())


# --- 3. 학습 및 테스트 데이터 분리 ---
# 데이터를 80%는 학습용, 20%는 검증용으로 분리
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)
print(f"\n학습 데이터: {X_train.shape}, 테스트 데이터: {X_test.shape}")


# --- 4. XGBoost 모델 생성 및 학습 ---
model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(label_encoder.classes_),
    eval_metric='mlogloss',
    use_label_encoder=False
)

print("\n--- 모델 학습 시작 ---")
model.fit(X_train, y_train)
print("--- 모델 학습 완료 ---")


# --- 5. 모델 성능 평가 ---
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\n모델 예측 정확도: {accuracy * 100:.2f}%")

report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print("\n--- 상세 평가 리포트 ---")
print(report)


# --- 6. 새로운 데이터 예측 예시 ---
# ❗️ 수정된 부분 1: 예측할 데이터에는 정답('Grade' 또는 '신용등급')을 포함하지 않습니다.
# ❗️ 수정된 부분 2: 'AT'와 'PD' 사이에 쉼표를 추가했습니다.
new_company_data = pd.DataFrame([{
    # 'corp_code'는 학습에서 제외했으므로 여기에서도 제외합니다.
    'CR': 1.74207198003175,
    'DER': 1.14099257484873,
    'ICR': 0,
    'OPM': 0.0407584511351937,
    'AT': 0.245063558668831, # 이 줄 끝에 쉼표가 없어서 추가했습니다.
    'PD': 0.175351018024796
    # 'Grade' 열은 예측 대상이므로 제거했습니다.
}])

# 새로운 데이터도 학습 데이터와 동일하게 인코딩
new_company_encoded = pd.get_dummies(new_company_data)
# 학습 시 사용된 열을 기준으로 재구성하여 열의 순서와 개수를 맞춥니다.
new_company_reindexed = new_company_encoded.reindex(columns=encoded_columns, fill_value=0)

# 예측 수행
prediction_encoded = model.predict(new_company_reindexed)

# 예측 결과(숫자)를 원래의 신용등급(문자)으로 변환
prediction_original = label_encoder.inverse_transform(prediction_encoded)

print(f"\n--- 새로운 기업 데이터 예측 결과 ---")
print(f"예측된 신용등급: {prediction_original[0]}")


--- 데이터 일부 확인 ---
   corp_code        CR       DER  ICR       OPM        AT        PD Grade
0     100601  1.075677  2.107564    0  0.037111  0.240051  0.094478     B
1     100939  1.907837  0.412088    0  0.053074  0.177101  0.000072    A0
2     101044  2.763931  0.283168    0 -0.474904  0.038977  0.000239    A0
3     101220  1.352805  1.140993    0  0.060384  0.315300  0.332609    CC
4     101257  1.742072  1.651780    0  0.001716  0.228900  0.081990     B

--- 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384 entries, 0 to 2383
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   corp_code  2384 non-null   int64  
 1   CR         2384 non-null   float64
 2   DER        2384 non-null   float64
 3   ICR        2384 non-null   int64  
 4   OPM        2384 non-null   float64
 5   AT         2384 non-null   float64
 6   PD         2384 non-null   float64
 7   Grade      2384 non-null   object 
dtypes: float64(

c:\Users\edukd\miniconda3\envs\kiwoombank\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:12:34] WARNING: D:\bld\xgboost-split_1737531313485\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
